### **1**

In [38]:
!pip install pyheif

In [39]:
import os
import numpy as np
import torch
import torch.utils.data
import pyheif
from PIL import Image
fscamera - 00060
fsduration - 00D 00H 00M 00S
class RandDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None, mode = 'train'):
        self.root = root
        self.transforms = transforms
        self.mode = mode
        # load all image files, sorting them to
        # ensure that they are aligned
        if self.mode == 'train':
            imgs_close = list(sorted(os.listdir(os.path.join(root, 'Train_closed_oyster_shells'))))
            lbl_close = np.zeros(len(imgs_close))
            imgs_open = list(sorted(os.listdir(os.path.join(root, 'Train_large_open_oyster_shells'))))
            lbl_open = np.ones(len(imgs_open))
        else:
            imgs_close = list(sorted(os.listdir(os.path.join(root, 'Test_closed_oyster_shells'))))
            lbl_close = np.zeros(len(imgs_close))
            imgs_open = list(sorted(os.listdir(os.path.join(root, 'Test_large_open_oyster_shells'))))
            lbl_open = np.ones(len(imgs_open))

        self.imgs = imgs_close + imgs_open
        self.labels = np.concatenate((lbl_close, lbl_open))

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        # load images ad masks
        if self.mode == 'train':
            if os.path.isfile(os.path.join(self.root, 'Train_closed_oyster_shells', self.imgs[idx])):
                imgs_path = os.path.join(self.root, 'Train_closed_oyster_shells', self.imgs[idx])

            if os.path.isfile(os.path.join(self.root, 'Train_large_open_oyster_shells', self.imgs[idx])):
                imgs_path = os.path.join(self.root, 'Train_large_open_oyster_shells', self.imgs[idx])
        
        else:
            if os.path.isfile(os.path.join(self.root, 'Test_closed_oyster_shells', self.imgs[idx])):
                imgs_path = os.path.join(self.root, 'Test_closed_oyster_shells', self.imgs[idx])

            if os.path.isfile(os.path.join(self.root, 'Test_large_open_oyster_shells', self.imgs[idx])):
                imgs_path = os.path.join(self.root, 'Test_large_open_oyster_shells', self.imgs[idx])

        heif_file = pyheif.read(imgs_path)
        img = Image.frombytes(
            heif_file.mode, 
            heif_file.size, 
            heif_file.data,
            "raw",
            heif_file.mode,
            heif_file.stride,
            )
        if self.transforms is not None:
            img = self.transforms(img)
        lbl = self.labels[idx]
        
        return img, torch.tensor(lbl, dtype=torch.long)   

SyntaxError: ignored

In [ ]:
#Data Transform
import torchvision.transforms as T


def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.Resize((224,224)))
    transforms.append(T.ToTensor())
    transforms.append(T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]))
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [40]:
dataset = RandDataset('Data', get_transform(train=True))
dataset_test = RandDataset('Data', get_transform(train=False),  mode = 'val')

NameError: ignored

In [41]:
print(len(dataset))
print(len(dataset_test))

NameError: ignored

In [ ]:
data_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True, pin_memory=True)
data_loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=32, shuffle=False)

NameError: ignored

In [42]:
# Inverse Transform
import matplotlib.pyplot as plt

inv_normalize = T.Normalize(
    mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
    std=[1/0.229, 1/0.224, 1/0.255]
)

def imshow(img):
    img = inv_normalize(img)     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

NameError: ignored

In [43]:
# Visualise Data Loader
import torchvision
imag, labl = next(iter(data_loader_test))
imshow(torchvision.utils.make_grid(imag))
print(labl)

ValueError: ignored

# 2

In [44]:
import torchvision
class ResNetBackbone34(torch.nn.Module):
    def __init__(self, out_filters=2):
        super(ResNetBackbone34, self).__init__()

        self.res = torchvision.models.resnet34(pretrained=True)
        fltr = self.res.fc.in_features
        self.res.fc = torch.nn.Linear(fltr, out_filters)

    def forward(self, x):
        x = self.res(x)
        return torch.nn.functional.softmax(x)

ValueError: ignored

In [45]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNetBackbone34()
model.to(device)

NameError: ignored

# 3


In [46]:
# Define the training parameters and function for optimise the parameters
epochs = 30
max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-4
val_acc = 0

loss_fn = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), max_lr, weight_decay=weight_decay)

sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, 
                                                steps_per_epoch=len(data_loader))

NameError: ignored

In [47]:
# Load and save the checkpoints
def save_checkpoint(save_path, model, val_loss, val_acc, start_epoch):

    if save_path == None:
        return
    
    state_dict = {'model_state_dict': model.state_dict(),
                  'val_loss': val_loss,
                  'val_acc' : val_acc,
                  'start_epoch' : start_epoch}
    
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')

def load_checkpoint(load_path, model, device='cpu'):
    
    if load_path==None:fswebcam -r 720x1280 3.jpg
        return
    
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    
    model.load_state_dict(state_dict['model_state_dict'])
    epoch = state_dict['start_epoch']
    return epoch

SyntaxError: ignored

In [48]:
# Training and Evaluaton
for epoch in range(epochs):
    model.train()
    train_losses = 0
    i = 1
    for batch in data_loader:
        print('{}/{} starting...'.format(i, len(data_loader)))
        i += 1
        img = batch[0].to(device)
        label = batch[1].to(device)

        optimizer.zero_grad()
        pred = model(img)

        loss = loss_fn(pred, label)
        loss.backward()

        #torch.nn.utils.clip_grad_value_(model.parameters(), grad_clip)
        optimizer.step()
        sched.step() 
        train_losses += loss.item()

    

    with torch.no_grad():
        correct = 0
        total = 0
        model.eval()
        val_loss = 0
        for batch in data_loader_test:
            img = batch[0].to(device)
            label = batch[1].to(device)

            pred = model(img)

            _, predicted = torch.max(pred.data, 1)
            total += label.size(0)
            correct += (predicted == label).sum().item()
            
            loss = loss_fn(pred, label)
            val_loss += loss.item()

        acc = 100 * (correct/total)
        val_loss = val_loss/len(data_loader_test)
        if acc>val_acc:
            save_checkpoint('weights_andr/epoch_'+str(epoch+1), model, val_loss, acc, epoch)
            val_acc = acc
    print('Epoch {}/{}, train_loss {}, valid_loss {}, valid_acc {}'.format(epoch+1, epochs, train_losses/len(data_loader), val_loss, acc))     


NameError: ignored

# 4

In [49]:
model = ResNetBackbone34()
load_checkpoint('/content/epoch_11', model)

model.to(device)

correct = 0
total = 0
#test_loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, pin_memory=True)
with torch.no_grad():
    model.eval()

    for batch in data_loader_test:
        img = batch[0].to(device)
        label = batch[1].to(device)

        pred = model(img)
        _, predicted = torch.max(pred.data, 1)
        print(predicted, label)
        total += label.size(0)
        correct += (predicted == label).sum().item()

print('accuracy is ', correct/total)

NameError: ignored

In [ ]:
!cp '/content/weights_andr/epoch_11' -r '/content/drive/My Drive/work/weights_andr'

cp: cannot stat '/content/weights_andr/epoch_11': No such file or directory


# 5

In [50]:
#download the model here
#https://drive.google.com/file/d/1zSm6cOwzhA4llN2cRZ63HJ6om4h34eD8/view?usp=sharing
!gdown --id 1zSm6cOwzhA4llN2cRZ63HJ6om4h34eD8

Permission denied: https://drive.google.com/uc?id=1zSm6cOwzhA4llN2cRZ63HJ6om4h34eD8
Maybe you need to change permission over 'Anyone with the link'?


In [ ]:
# Load the trained model
model = ResNetBackbone34()
state_dict = torch.load("/content/epoch_11", torch.device("cpu"))
model.load_state_dict(state_dict['model_state_dict'])

NameError: ignored

In [ ]:
from google.colab import files
uploaded = files.upload()
file_names, _ = list(uploaded.items())[0]

TypeError: ignored

In [ ]:
#Test just update the file_names 
from PIL import Image
import pyheif

file_names = '/content/Data/Test_closed_oyster_shells/IMG_0685.HEIC'

heif_file = pyheif.read(file_names)
image = Image.frombytes(
    heif_file.mode, 
    heif_file.size, 
    heif_file.data,
    "raw",
    heif_file.mode,
    heif_file.stride,
    )

trans_fn = get_transform(train=False)
test_image = trans_fn(image)
test_image = test_image.unsqueeze(dim=0)
pred = model(test_image.to(device))
_, predicted = torch.max(pred.data, 1)
predicted

In [ ]:
classes = ['Close', 'Open']
print('Predicted class is ',classes[predicted.item()])